# Grid World Environment formulized using MDPs

In [37]:
import numpy as np

class GridWorldEnv:
    def __init__(self):
        self.grid_size = (3, 4) # gird world
        self.start_state = (0, 0) # start state
        self.goal_state = (2, 3)  # goal state
        self.fire_state = (1, 3)  # fire state
        self.null_space = (1, 1)  # null space
        
        self.step_reward = 0  # reward for each step
        self.fire_penalty = -1  # penalty for fire
        self.goal_reward = 1  # reward for reaching the goal
        
        self.intended_prob = 0.8  # probability of taking the intended action
        self.other_prob = (1 - self.intended_prob) / 3  # probability of taking a random action
        
        self.actions = ["up", "down", "left", "right"]  # available actions
        
        self.reset()
    
    # Reset the environment
    def reset(self):
        self.state = self.start_state
    
    # State transition function
    def step(self, action):
        if self._is_terminal(self.state):
            return self.state, 0, True, {}
        
        i, j = self.state
        next_state = self._get_next_state(i, j, action) # get next state based on the action
        
        reward = self._get_reward(next_state) # get reward based on the next state
        
        done = self._is_terminal(next_state) # check if the task is completed
        
        self.state = next_state
        return next_state, reward, done, {}

    # State transition helper function
    def _get_next_state(self, i, j, action):
        moves = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}
        chosen_action = action if np.random.rand() < self.intended_prob else np.random.choice([a for a in self.actions if a != action])
        print
        di, dj = moves[chosen_action]
        next_state = (i + di, j + dj)
        
        return next_state if self._is_valid_state(next_state) else (i, j)
    
    # Render the environment
    def render(self):
        grid = np.full(self.grid_size, 'O', dtype=str)
        grid[self.start_state] = 'S'
        grid[self.goal_state] = 'G'
        grid[self.fire_state] = '!'
        grid[self.null_space] = 'X'
        
        state_i, state_j = self.state
        if grid[state_i, state_j] in {'O', '!', 'G'}:
            grid[state_i, state_j] = '@'  # Agent's position
        
        # print("+" + "---+" * self.grid_size[1])
        for row in grid:
            print(" ".join(row))
            # print("| " + " | ".join(row) + " |")
            # print("+" + "---+" * self.grid_size[1])
        print()
    
    def get_all_states(self):
        states = []
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                state = (i, j)
                if self._is_valid_state(state):
                    states.append(state)
        return states
    
    #  Task completion check
    def _is_terminal(self, state):
        return state == self.goal_state
    
    # Environment bound check
    def _is_valid_state(self, state):
        i, j = state
        return 0 <= i < self.grid_size[0] and 0 <= j < self.grid_size[1] and state != self.null_space
    
    # Reward function
    def _get_reward(self, state):  
        if state == self.fire_state:
            return self.fire_penalty
        elif state == self.goal_state:
            return self.goal_reward
        else:
            return self.step_reward

In [43]:
# Example usage
env = GridWorldEnv()

print("Initial State:")
env.reset()
env.render()

for i in range (4):
    action = "down"
    next_state, reward, done, _ = env.step(action)
    print(f"Action: {action}, Reward: {reward}, Next State: {next_state}")
    env.render()

Initial State:
S O O O
O X O !
O O O G

Action: down, Reward: 0, Next State: (1, 0)
S O O O
@ X O !
O O O G

Action: down, Reward: 0, Next State: (2, 0)
S O O O
O X O !
@ O O G

Action: down, Reward: 0, Next State: (2, 0)
S O O O
O X O !
@ O O G

Action: down, Reward: 0, Next State: (2, 0)
S O O O
O X O !
@ O O G



# Solution Methods

## Random steps

In [11]:
env.reset()
done = False
n = 0
while not done:
    action = env.actions[np.random.choice(len(env.actions))]
    next_state, reward, done, _ = env.step(action)
    print(f"Step:{n} -  Action: {action}, Reward: {reward}, Next State: {next_state}")
    n += 1
    env.render()

Step:0 -  Action: up, Reward: 0, Next State: (0, 0)
S O O O
O X O F
O O O G

Step:1 -  Action: left, Reward: 0, Next State: (0, 0)
S O O O
O X O F
O O O G

Step:2 -  Action: down, Reward: 0, Next State: (1, 0)
S O O O
A X O F
O O O G

Step:3 -  Action: up, Reward: 0, Next State: (0, 0)
S O O O
O X O F
O O O G

Step:4 -  Action: right, Reward: 0, Next State: (0, 1)
S A O O
O X O F
O O O G

Step:5 -  Action: down, Reward: 0, Next State: (0, 1)
S A O O
O X O F
O O O G

Step:6 -  Action: left, Reward: 0, Next State: (0, 1)
S A O O
O X O F
O O O G

Step:7 -  Action: right, Reward: 0, Next State: (0, 2)
S O A O
O X O F
O O O G

Step:8 -  Action: right, Reward: 0, Next State: (0, 3)
S O O A
O X O F
O O O G

Step:9 -  Action: right, Reward: 0, Next State: (0, 3)
S O O A
O X O F
O O O G

Step:10 -  Action: down, Reward: 0, Next State: (0, 2)
S O A O
O X O F
O O O G

Step:11 -  Action: right, Reward: 0, Next State: (1, 2)
S O O O
O X A F
O O O G

Step:12 -  Action: up, Reward: 0, Next State: (0,

## Value Iteration

https://gibberblot.github.io/rl-notes/single-agent/value-iteration.html

```python
# Step 1: Initialize the value function
V = {state: 0 for state in env.get_all_states()}

# Step 2: Define hyperparameters
gamma = 0.99
theta = 1e-6

# Step 3: Perform value iteration
while True:
    delta = 0
    
    for state in env.get_all_states():
        if env.is_terminal(state):
            continue
        
        v = V[state]
        max_value = float('-inf')
        
        for action in env.get_possible_actions(state):
            expected_value = sum(prob * (reward + gamma * V[next_state])
                                 for next_state, prob, reward in env.get_transitions(state, action))
            
            max_value = max(max_value, expected_value)
        
        V[state] = max_value
        delta = max(delta, abs(v - V[state]))
    
    if delta < theta:
        break

# Step 4: Extract the optimal policy
policy = {}

for state in env.get_all_states():
    if env.is_terminal(state):
        continue
    
    best_action = None
    max_value = float('-inf')
    
    for action in env.get_possible_actions(state):
        expected_value = sum(prob * (reward + gamma * V[next_state])
                             for next_state, prob, reward in env.get_transitions(state, action))
        
        if expected_value > max_value:
            max_value = expected_value
            best_action = action
    
    policy[state] = best_action
```


In [495]:
env.reset()
value_function = np.zeros(env.grid_size)
gamma = 0.9

n = 0
while n < 1:
    for state in env.get_all_states():

        if env._is_terminal(state):
            continue

        v = value_function[state]
        
        next_values = []
        for action in env.actions:
            next_state, reward, _, _ = env.step(action)
            next_values.append(reward + gamma * value_function[next_state])
            print(f"State: {state}, Action: {action}, Next State: {next_state}, Reward: {reward}")
            print(f"{value_function}")
            env.render()
            
        value_function[state] = max(next_values)
        env.reset()
    print(value_function)
    n += 1


State: (0, 0), Action: up, Next State: (0, 0), Reward: 0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
S O O O
O X O F
O O O G

State: (0, 0), Action: down, Next State: (0, 0), Reward: 0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
S O O O
O X O F
O O O G

State: (0, 0), Action: left, Next State: (0, 0), Reward: 0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
S O O O
O X O F
O O O G

State: (0, 0), Action: right, Next State: (0, 1), Reward: 0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
S A O O
O X O F
O O O G

State: (0, 1), Action: up, Next State: (0, 0), Reward: 0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
S O O O
O X O F
O O O G

State: (0, 1), Action: down, Next State: (0, 0), Reward: 0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
S O O O
O X O F
O O O G

State: (0, 1), Action: left, Next State: (0, 0), Reward: 0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
S O O O
O X O F
O O O G

State: (0, 1), Action: right, Next State: (1, 0), Reward: 0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0